In [146]:
import os
import pandas as pd 

In [147]:
%ls clean_csv/

L3_counts_per_user/  raw_L3counts_data/


In [148]:
main_transcript_counts_path = './end-als/transcriptomics-data/L3_counts/'
data_counts_list = os.listdir(main_transcript_counts)

In [149]:
for folder in data_counts_list:
    # Get the subfolder name into the main folder 
    sub_folder =  os.listdir(main_transcript_counts_path + folder + '/')[0]
    # Get the file name into sub folder 
    sub_sub_folder = os.listdir(main_transcript_counts_path + folder + '/' + sub_folder)[0]
    # Generating complete path of dile 
    complete_path = main_transcript_counts_path + folder + '/' + sub_folder + '/' + sub_sub_folder
    # Read csv file 
    transcript_dataframe = pd.read_csv(complete_path, delimiter='\t')
    # Get the unic column name
    column_name = transcript_dataframe.columns[0]
    # reset index to delete levels in trascript data set
    transcript_dataframe = transcript_dataframe.reset_index()
    # Rename columns geneid and transcriptopic counts
    transcript_dataframe = transcript_dataframe.rename(columns={column_name: folder, "level_0":"Geneid"})
    # Take only genId and transcript data from the dataframe
    transcript_dataframe= transcript_dataframe[["Geneid", folder]]
    # Drop the first row, this row contains the real name of the columns and is not neccesary 
    transcript_dataframe = transcript_dataframe.drop(0, axis=0).reset_index(drop=True)
    transcript_dataframe.to_csv('./clean_csv/L3_counts_per_user/' + folder + '.csv', index=False)
    

In [150]:
cleaning_L3counts_path = './clean_csv/L3_counts_per_user/'
dataset_list = os.listdir(cleaning_L3counts_path)
all_transcript_data = []
flag = True

In [151]:
for name in dataset_list:
    individual_trascript_data = pd.read_csv(cleaning_L3counts_path + name)
    if (flag):
        all_transcript_data  = individual_trascript_data
        flag= False
        continue
    all_transcript_data = all_transcript_data.merge(individual_trascript_data, how='left', on='Geneid')
all_transcript_data = all_transcript_data.reset_index(drop=True) 
all_transcript_data.to_csv('./clean_csv/raw_L3counts_data/raw_L3counts_data.csv', index=False)

In [242]:
# Read Raw count data
raw_counts_data = pd.read_csv('./clean_csv/raw_L3counts_data/raw_L3counts_data.csv')
raw_counts_data = pd.pivot_table(raw_counts_data,values= raw_counts_data.columns, columns=['Geneid'] ).reset_index().rename(columns={"index":"Participant_ID"})


In [243]:
# Read clinical data portal to merge with quantile normalization df
clinical_dataportal = pd.read_csv('/home/jupyter/end-als/clinical-data/filtered-metadata/metadata/aals_dataportal_datatable.csv', 
                                  usecols=['Participant_ID', 'Sex', 'Ethnicity', 'Race', 'Site of Onset', 'Age at Sample Collection', 
                                           'Age At Symptom Onset', 'Subject Group']).rename(columns={"Site of Onset":"SiteOnset", "Age at Sample Collection":"Age", 
                                                                                                     "Age At Symptom Onset":"AgeOnSet", 'Subject Group':'subjectGroup'})
clinical_dataportal['SiteOnset'] = clinical_dataportal['SiteOnset'].fillna('Healthy Control')
all_grouping_transcript_data = clinical_dataportal.merge(raw_counts_data, how='right', on='Participant_ID')
all_grouping_transcript_data = all_grouping_transcript_data.sort_values(by=['Participant_ID']).reset_index(drop=True)
all_grouping_transcript_data['Age'] = all_grouping_transcript_data['Age'].fillna(0)
age_list = []

for index in all_grouping_transcript_data.index:
    age = all_grouping_transcript_data.loc[index, 'Age']
    if (age == 0):
        age = all_grouping_transcript_data.loc[index, 'AgeOnSet']
    if(age < 18 ):
        age_list.append('18-')
        continue
    if(age >= 18 and age <= 24):
        age_list.append('18-24')
        continue
    if(age >= 25 and age <= 34):
        age_list.append('25-34')
        continue
    if(age >= 35 and age <= 44):
        age_list.append('35-44')
        continue
    if(age >= 45 and age <= 54):
        age_list.append('45-54')
        continue
    if(age >= 55 and age <= 64):
        age_list.append('55-64')
        continue
    if(age >= 65 ):
        age_list.append('65+')
        continue

all_grouping_transcript_data['Age'] = age_list

In [244]:
all_grouping_transcript_data[["Participant_ID", "Sex","Ethnicity", "Race", "SiteOnset", "Age", 'subjectGroup']].to_csv('./clean_csv/raw_L3counts_data/transcript_data_groups.csv')
all_grouping_transcript_data[["Participant_ID", "Sex","Ethnicity", "Race", "SiteOnset", "Age", "AgeOnSet", 'subjectGroup']]

,Participant_ID,Sex,Ethnicity,Race,SiteOnset,Age,AgeOnSet,subjectGroup
0,CASE-NEUAA599TMX,Female,Not Hispanic or Latino,White,Bulbar,65+,77.0,ALS
1,CASE-NEUAB000NKC,Female,Not Hispanic or Latino,White,Limb,45-54,50.0,ALS
2,CASE-NEUAE993EPR,Female,Not Hispanic or Latino,Black/African American,Multiple,55-64,55.0,ALS
3,CASE-NEUAF553MJ3,Female,Not Hispanic or Latino,White,Bulbar,65+,59.0,ALS
4,CASE-NEUAG603XLK,Female,Not Hispanic or Latino,White,Limb,55-64,54.0,ALS
...,...,...,...,...,...,...,...,...
159,CTRL-NEUUV825HYF,Female,Not Hispanic or Latino,White,Healthy Control,65+,NaN,Healthy Control
160,CTRL-NEUVZ050YX7,Male,Not Hispanic or Latino,White,Healthy Control,45-54,NaN,Healthy Control
161,CTRL-NEUWT164JRQ,Female,Not Hispanic or Latino,White,Healthy Control,65+,NaN,Healthy Control
162,CTRL-NEUYM205MRL,Male,Not Hispanic or Latino,White,Healthy Control,55-64,NaN,Healthy Control


In [241]:
all_grouping_transcript_data

,Participant_ID,Sex,Ethnicity,Race,SiteOnset,AgeOnSet,Age,ENSG00000000003,ENSG00000000005,ENSG00000000419,...,ENSG00000283098,ENSG00000283101,ENSG00000283103,ENSG00000283108,ENSG00000283110,ENSG00000283117,ENSG00000283118,ENSG00000283122,ENSG00000283123,ENSG00000283125
0,CASE-NEUAA599TMX,Female,Not Hispanic or Latino,White,Bulbar,77.0,65+,30,72,639,...,2,0,222,2,0,0,0,69,0,0
1,CASE-NEUAB000NKC,Female,Not Hispanic or Latino,White,Limb,50.0,45-54,937,2,774,...,3,0,289,15,0,3,0,110,0,0
2,CASE-NEUAE993EPR,Female,Not Hispanic or Latino,Black/African American,Multiple,55.0,55-64,1044,0,867,...,3,0,533,0,0,1,0,29,0,0
3,CASE-NEUAF553MJ3,Female,Not Hispanic or Latino,White,Bulbar,59.0,65+,662,10,643,...,2,0,439,8,0,5,0,52,0,0
4,CASE-NEUAG603XLK,Female,Not Hispanic or Latino,White,Limb,54.0,55-64,421,0,314,...,2,0,204,0,0,3,0,24,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,CTRL-NEUUV825HYF,Female,Not Hispanic or Latino,White,Healthy Control,NaN,65+,767,2,456,...,3,0,262,0,0,1,0,52,0,0
160,CTRL-NEUVZ050YX7,Male,Not Hispanic or Latino,White,Healthy Control,NaN,45-54,1029,7,568,...,0,0,346,0,0,1,0,165,0,0
161,CTRL-NEUWT164JRQ,Female,Not Hispanic or Latino,White,Healthy Control,NaN,65+,1230,4,1017,...,2,0,379,0,0,6,0,75,0,0
162,CTRL-NEUYM205MRL,Male,Not Hispanic or Latino,White,Healthy Control,NaN,55-64,907,0,623,...,4,0,283,0,0,3,0,59,0,0


In [237]:
all_grouping_transcript_data = all_grouping_transcript_data.drop(["Sex","Ethnicity", "Race", "SiteOnset", "Age", "AgeOnSet"], axis=1)

In [238]:
all_grouping_transcript_data = pd.pivot_table(all_grouping_transcript_data, values=all_grouping_transcript_data.columns, columns=['Participant_ID'])
all_grouping_transcript_data.to_csv('./clean_csv/raw_L3counts_data/raw_L3counts__ordered_data.csv')

Participant_ID,CASE-NEUAA599TMX,CASE-NEUAB000NKC,CASE-NEUAE993EPR,CASE-NEUAF553MJ3,CASE-NEUAG603XLK,CASE-NEUAM655HF7,CASE-NEUAT234RK6,CASE-NEUAW157NMJ,CASE-NEUAW717TN6,CASE-NEUAX665ZHY,...,CTRL-NEUMT184NWC,CTRL-NEUNW343RXP,CTRL-NEUPH301NNX,CTRL-NEUPL878MTL,CTRL-NEURJ861MMD,CTRL-NEUUV825HYF,CTRL-NEUVZ050YX7,CTRL-NEUWT164JRQ,CTRL-NEUYM205MRL,CTRL-NEUZL045YD3
ENSG00000000003,30.0,937.0,1044.0,662.0,421.0,879.0,784.0,965.0,628.0,1161.0,...,881.0,1200.0,714.0,1126.0,715.0,767.0,1029.0,1230.0,907.0,886.0
ENSG00000000005,72.0,2.0,0.0,10.0,0.0,3.0,0.0,0.0,1.0,6.0,...,0.0,1.0,0.0,0.0,1.0,2.0,7.0,4.0,0.0,0.0
ENSG00000000419,639.0,774.0,867.0,643.0,314.0,590.0,784.0,760.0,592.0,902.0,...,723.0,744.0,483.0,923.0,624.0,456.0,568.0,1017.0,623.0,985.0
ENSG00000000457,445.0,257.0,715.0,563.0,256.0,477.0,579.0,545.0,557.0,828.0,...,689.0,539.0,466.0,831.0,525.0,388.0,474.0,664.0,465.0,687.0
ENSG00000000460,286.0,168.0,269.0,254.0,94.0,197.0,308.0,240.0,286.0,293.0,...,263.0,232.0,220.0,251.0,256.0,198.0,242.0,357.0,186.0,242.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283117,0.0,3.0,1.0,5.0,3.0,0.0,3.0,3.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,3.0,0.0
ENSG00000283118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000283122,69.0,110.0,29.0,52.0,24.0,57.0,63.0,83.0,57.0,35.0,...,45.0,81.0,28.0,57.0,42.0,52.0,165.0,75.0,59.0,44.0
ENSG00000283123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
